In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase, str_rep_vdb_ing, str_rep_vdb_nam 
import pandas as pd
additional_cbr_1, additional_cbr_2, additional_validation, cbr_augmentation, cbr_database, model_training, validation = load_all_datasets()

DATABASE_NAME = "cbr_database"
OUTPUT_FILE = "validation_cbr_00_gpt2_input.csv"
cbr_id = cbr_database.set_index("id")

In [2]:
# large_cbr = pd.concat([cbr_database, cbr_augmentation, additional_cbr_1, additional_cbr_2])
# large_cbr_id = large_cbr.set_index("id")
# large_cbr_reps = str_rep_vdb(large_cbr_id)
# len(large_cbr_reps)

In [3]:
# meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_id["id"].values)], {"dataset":"cbr_database"})
# vdb = CBRDatabase(large_cbr_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)

vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)
print(len(vdb.get_texts()))
print(vdb.similarity_search(query="test", k=1))

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
10627
[Document(page_content='name: jalapeo cheese and smoked paprika bread bread machine ; ingredients: water, bread flour, jalapeno cheddar cheese, smoked paprika, garlic clove, sugar, salt, active dry yeast ; preparation:', metadata={'dataset': 'cbr_database', 'recipe_id': 244278})]


In [4]:
# generowanie inputów - 1500 tokenów 
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in validation.iterrows():
    while True:
        # recipe_ixs = [random.randint(0, len(cbr_database)-1) for _ in range(7)]
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=5)]
        # recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query=v_recipe["ingredients"], k=7)]
        # recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query=v_recipe["name"], k=7)]
        # print(vdb.similarity_search(query=v_recipe["ingredients"], k=7))#dev
        # print(large_cbr_id.loc[recipe_ixs])#dev

        gen_input = compose(cbr_id.loc[recipe_ixs], v_recipe)
        br = False
        if len(tokenizer.encode(gen_input)) <= 1024: 
            br = True
        else:
            for ii in list(range(6, -1, -1)):
                gen_input = compose(cbr_id.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 1024: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            if br: break
            for ii in list(range(5, -1, -1)):
                gen_input = compose(cbr_id.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 1024: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_id.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 - 1292


In [5]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}", index=False)

In [6]:
vdb.similarity_search(query="str_rep_no_steps(v_recipe)", k=7)

[Document(page_content='name: v s crock pot bottom round roast ; ingredients: boneless bottom round roast, dijon mustard, brown sugar, worcestershire sauce, dry red wine, beef broth, onion, carrots, kosher salt & freshly ground black pepper ; preparation:', metadata={'dataset': 'cbr_database', 'recipe_id': 276903}),
 Document(page_content='name: holiday vegan golden gravy ; ingredients: flour, nutritional yeast flakes, oil, water, soy sauce, salt & pepper ; preparation:', metadata={'dataset': 'cbr_database', 'recipe_id': 35838}),
 Document(page_content='name: savory slow cooker beef roast ; ingredients: boneless beef roast, onion, vegetable juice, brown gravy mix, garlic cloves, pepper, thyme ; preparation:', metadata={'dataset': 'cbr_database', 'recipe_id': 257081}),
 Document(page_content='name: vegan caesar salad dressing ; ingredients: dijon mustard, nutritional yeast flakes, almonds, garlic cloves, water, lemon juice, soy sauce, olive oil, white pepper ; preparation:', metadata={'